In [ ]:
! pip install git+https://github.com/openai/whisper.git -q

     |████████████████████████████████| 5.5 MB 27.8 MB/s 
     |████████████████████████████████| 182 kB 68.5 MB/s 
     |████████████████████████████████| 7.6 MB 47.4 MB/s 


In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import ipywidgets as widgets

languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="ko_kr",
    description='Language:',
    disabled=False,
)

selection

Dropdown(description='Language:', index=39, options=(('Select language', None), ('----------', None), ('Afrika…

In [ ]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Malayalam (ml_in)


In [ ]:
class Fleurs(torch.utils.data.Dataset):
    """
    A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
    """
    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            with urllib.request.urlopen(url) as source, open(tar_path, "wb") as output:
                with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
                    while True:
                        buffer = source.read(8192)
                        if not buffer:
                            break

                        output.write(buffer)
                        loop.update(len(buffer))

        labels = {}
        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]
                    

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]
        
        return (audio, text)

In [ ]:
dataset = Fleurs(lang, subsample_rate=10)  # subsample 10% of the dataset for a quick demo

  0%|                                              | 0.00/2.68G [00:00<?, ?iB/s]

In [ ]:
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 762,321,920 parameters.


In [ ]:
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)

In [ ]:
references = []
transcriptions = []
translations = []

for audio, text in tqdm(dataset):
    transcription = model.transcribe(audio, **transcribe_options)["text"]
    translation = model.transcribe(audio, **translate_options)["text"]
    
    transcriptions.append(transcription)
    translations.append(translation)
    references.append(text)

In [ ]:
data = pd.DataFrame(dict(reference=references, transcription=transcriptions, translation=translations))
data

In [ ]:
! pip install jiwer

In [ ]:
import jiwer
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

In [ ]:
data["transcription_clean"] = [normalizer(text) for text in data["transcription"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data

,reference,transcription,translation,transcription_clean,reference_clean
0,எங்களுடைய விளையாட்டு வீரர்கள் மற்றும் கிளப்புகளின் நலன்கள் மற்றும் அவர்களின் விளையாட்டு ஆகியவை தீர்மானிப்பதை விட எங்கள் நிறுவனத்திற்குள் அர்த்தமுள்ள மாற்றத்துடன் முன்னேறுவதன் மூலம் சிறப்பாக சேவை செய்யப்படலாம் என்ற usoc's அறிக்கையுடன் நாங்கள் உடன்படுகிறோம்,எங்களுடைய விளையாட்டு விரார்கள் மட்டும் கிலைப்புகளை நலைங்கள் மட்டும் அவர்களுக்கு விளையாட்டு ஆகியவை தீர்மானிப்பதை விட எங்கள் நிறுவனத்திற்கு அர்த்தமுள்ள மாற்றத்துடன் முன் நிறுவனத்தன் முழும் சிறப்பாக சேவை செயலாம் எங்களுடைய வியாட்டுகள் மட்டும் கிலைப்புகளை நிறுவனத்திற்கு அர்த்தமுள் மட்டும் கிலைப்பாக சேவை செயலாம்,"Our najat for stray cats and flycats, and our",எஙகள ட ய வ ள ய டட வ ர ரகள மடட ம க ல பப கள நல ஙகள மடட ம அவரகள கக வ ள ய டட ஆக யவ தரம ன பபத வ ட எஙகள ந ற வனதத றக அரததம ளள ம றறதத டன ம ன ந ற வனததன ம ழ ம ச றபப க ச வ ச யல ம எஙகள ட ய வ ய டட கள மடட ம க ல பப கள ந ற வனதத றக அரததம ள மடட ம க ல பப க ச வ ச யல ம,எஙகள ட ய வ ள ய டட வரரகள மறற ம க ளபப கள ன நலனகள மறற ம அவரகள ன வ ள ய டட ஆக யவ தரம ன பபத வ ட எஙகள ந ற வனதத றக ள அரததம ளள ம றறதத டன ம னன ற வதன ம லம ச றபப க ச வ ச யயபபடல ம எனற usoc is அற கக ய டன ந ஙகள உடனபட க ற ம
1,தீவுக்கூட்டங்களிலும் ஏரிகளிலும் படகுகள் அவசியமானதல்ல,தீவுக்கூட்டங்களிலும் ஏறிகளிலும் படகுகள் அவசியமானது அல்ல.,"do not leave the boat animals or animals are not observed as passersby learn from them share, like, comment and share See u in the next video...",தவ கக டடஙகள ல ம ஏற கள ல ம படக கள அவச யம னத அலல,தவ கக டடஙகள ல ம ஏர கள ல ம படக கள அவச யம னதலல
2,சர்ச்களின் மத்திய அதிகாரம் கடந்த ஆயிரம் ஆண்டுகளுக்கு மேலாக ரோமில் இருந்து வருகிறது மற்றும் இந்த அதிகாரம் மற்றும் செல்வத்தின் ஒரே இடத்திலான செறிவு பல பேரை அடிப்படைக் கோட்பாடுகள் நிறைவேற்றப் படுகின்றனவா என்ற கேள்விக்கு இட்டுச் செல்கிறது,சர்ச்சுகளின் மத்திய அதிகாரம் கடந்த ஆயிரம் ஆண்டுகளுக்கு மேலாக ரோமிலிருந்து வரையது மற்றும் இந்த அதிகாரம் மற்றும் செல்வத்தின் ஒரே இடத்திலான செரிவு பல பேரை அழிப்படை கோட்பாடுகள் நிறைவேற படிக்கின்றனவா என்ற கேள்விக்கு இட்டுச் செல்கிறது,"Churches and the middle authorities have been coming from Rome for more than a thousand years The question is, should the rule of law and the rule of law be followed in the same place as the rule of law and the rule of law?",சரசச கள ன மதத ய அத க ரம கடநத ஆய ரம ஆணட கள கக ம ல க ர ம ல ர நத வர யத மறற ம இநத அத க ரம மறற ம ச லவதத ன ஒர இடதத ல ன ச ர வ பல ப ர அழ பபட க டப ட கள ந ற வ ற பட கக னறனவ எனற க ளவ கக இடட ச ச லக றத,சரசகள ன மதத ய அத க ரம கடநத ஆய ரம ஆணட கள கக ம ல க ர ம ல இர நத வர க றத மறற ம இநத அத க ரம மறற ம ச லவதத ன ஒர இடதத ல ன ச ற வ பல ப ர அட பபட க க டப ட கள ந ற வ றறப பட க னறனவ எனற க ளவ கக இடட ச ச லக றத
3,கோயத் ஃபிக்டே மற்றும் ஸ்க்லீகல் போன்ற எழுத்தாளர்களிடமிருந்து பெறப்பட்ட கலாச்சார உறுதிக்கான மிகப் பெரிய அம்சம் ரொமான்டிசிஸத்தில் இருந்தது,கோயத் திக்டே மற்றும் சுக்கிகள் போன்ற எழுத்தாளர்களில மிருந்து பிறப்பட்ட கலாசார உறுதிக்கான மிகப்பெரிய அம்சம் ரொமாண்டிச்சத்தில் இருந்தது,"In the Romantic era, a large part of the culture developed by the writers like Goyath, Fikde and Skli was in the Romantic era.",க யத த கட மறற ம ச கக கள ப னற எழ தத ளரகள ல ம ர நத ப றபபடட கல ச ர உற த கக ன ம கபப ர ய அமசம ர ம ணட சசதத ல இர நதத,க யத ஃப கட மறற ம ஸகலகல ப னற எழ தத ளரகள டம ர நத ப றபபடட கல சச ர உற த கக ன ம கப ப ர ய அமசம ர ம னட ச ஸதத ல இர நதத
4,இதன் விளைவாக ஒரு தடையை கடக்க ஒன்றிணைந்து வேலை செய்யும் ஒரு நிறுவனத்தின் செயல்முறையானது வாடிக்கையாளரின் தேவையை பூர்த்தி செய்ய ஒரு புதிய புதுமையான செயல்முறைக்கு வழிவகுக்கலாம்,இதன் விளைவாக ஒரு தரைகிக் கடக்கு ஒன்றுநில் வேலை செய்யும் ஒரு நிறுவனத்தின் செயல் முழையானது இதன் விளைவாக ஒரு தரைகிக் கடக்கு ஒன்றுநில் வேலை செய்யும் ஒரு நிறுவனத்தின் செயல் முழையானது,It is a company that works from one place to another. It is a company that works from one place to another.,இதன வ ள வ க ஒர தர க க கடகக ஒனற ந ல வ ல ச யய ம ஒர ந ற வனதத ன ச யல ம ழ ய னத இதன வ ள வ க ஒர தர க க கடகக ஒனற ந ல வ ல ச யய ம ஒர ந ற வனதத ன ச யல ம ழ ய னத,இதன வ ள வ க ஒர தட ய கடகக ஒனற ண நத வ ல ச யய ம ஒர ந ற வனதத ன ச யலம ற ய னத வ ட கக ய ளர ன த வ ய ப ரதத ச யய ஒர ப த ய ப த ம ய ன ச யலம ற கக வழ வக ககல ம
5,1963 இல் அணை கட்டப்பட்ட பிறகு ஆறு முழுவதும் படிவுகளைப்

In [ ]:
wer = jiwer.wer(list(data["reference_clean"]), list(data["transcription_clean"]))
print(f"WER: {wer * 100:.2f} %")

WER: 33.63 %
